<a href="https://colab.research.google.com/github/37stu37/Hikurangi_mhzrd/blob/master/mhzrd_hikurangi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import os
import random

In [0]:
# raw data
earthquake_csv = pd.read_excel('/content/drive/My Drive/04_Cloud/01_Work/GNS/SSIF_2019/6 Flagship_Hikurangi/tsunami_bathtub_model/SubductionSources_27June2019.xls')
tsunami_csv = pd.read_csv('/content/drive/My Drive/04_Cloud/01_Work/GNS/SSIF_2019/6 Flagship_Hikurangi/tsunami_bathtub_model/BTable_20km_Hpct99_27June2019.csv')

earthquake_sources = earthquake_csv.UID.to_list()

In [0]:
earthquake_csv.head(1)

In [0]:
### test
# for i in range(1):
#   e = random.choice(earthquake_sources)
#   print(type(e))



---

Functions


---




In [0]:
class Earthquake:

    def __init__(self, name):
        self.name = name

    def EarthquakeSource():
        # select a source area to trigger a rupture
        e = random.choice(earthquake_sources)
        # crop dataframe to the selected source
        earthquake_df = earthquake_csv[['UID', 'B-value - pref', 'a-value']] # ! add "a" value later on
        earthquake_df = earthquake_df[earthquake_df.UID = e]
        # create the discrete frequency / magnitude distribution from the source selected
        magnitudes = np.arange(0, 11, 0.1)
        frequencies = 10**(earthquake_df['a_value']-(earthquake_df['B-value - pref']*magnitudes))
        frequencies = frequencies / frequencies.sum() # make the probability of having an earthquake from 0 to 1
        # select a random magnitude from the distribution 
        magnitude_RNG = np.random.choice(magnitudes, 1, p=frequencies)
        # (back)find the recurrence time generated
        for i in [i for i,x in enumerate(magnitudes) if x == magnitudes_RNG]:
            rec_time = frequencies[i]
        # create dictionary for the event v=created
        uid = earthquake_df.UID
        magnitude = magnitude_RNG[0]
        rec_time = rec_time
        # event_parameters = {'UID': earthquake_df.UID, 'Magnitude': magnitude_RNG[0], 'Recurrence_time': rec_time}
        # return earthquake_parameters # use dict['Recurrence_time] to get the details from the earthquake generated




def EarthquakeSource():
    # select a source area to trigger a rupture
    e = random.choice(earthquake_sources)
    # crop dataframe to the selected source
    earthquake_df = earthquake_csv[['UID', 'B-value - pref', 'a-value']] # ! add "a" value later on
    earthquake_df = earthquake_df[earthquake_df.UID = e]
    # create the discrete frequency / magnitude distribution from the source selected
    magnitudes = np.arange(0, 11, 0.1)
    frequencies = 10**(earthquake_df['a_value']-(earthquake_df['B-value - pref']*magnitudes))
    frequencies = frequencies / frequencies.sum() # make the probability of having an earthquake from 0 to 1
    # select a random magnitude from the distribution 
    magnitude_RNG = np.random.choice(magnitudes, 1, p=frequencies)
    # (back)find the recurrence time generated
    for i in [i for i,x in enumerate(magnitudes) if x == magnitudes_RNG]:
        rec_time = frequencies[i]
    # create dictionary for the event v=created
    event_parameters = {'UID': earthquake_df.UID, 'Magnitude': magnitude_RNG[0], 'Recurrence_time': rec_time}
    return earthquake_parameters # use dict['Recurrence_time] to get the details from the earthquake generated


def EarthquakeShaking():
    # get the details from the source earthquake


In [0]:
def Landslide():
    # get the shaking at node location
    # get the landslide suscpetibility
    # sample from the area / volume relationship
    # get a volume and runout + target node

In [0]:
def Tsunami():
    # where is the source ?
    # calculate Ht for all target areas (Bij) using the Earthquake Mw Ht = 10^Mw-Bij
    # from the distance to shore value, calculate the Water depth Wd = (Ht*2) - (distance from shore in meters/400)

1) random selection of the source earthquake location & sampling of the probability / intensity distribution (N = 10^(a-bMw))

2) generation of tsunami based on the Bij value (Ht = 10^(Mw-Bij) & calculation of depth of inundation (Htx2 - decay)